In [0]:
!pip install xgboost

In [0]:
#Carregando as Bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_selection import chi2, SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, mean_absolute_error

###Importando os Dados da camada Bronze
- Criando o DF com Pandas
- Mostrando os primeiros registros

In [0]:
#Importando o caminho para o dataset na Bronze
url_dataset = '/Volumes/workspace/bronze/car_insurance_claim/Car_Insurance_Claim.csv'

#Criando o DataFrame
df_insurance = pd.read_csv(url_dataset)
display(df_insurance.head())

## 1) **Aplicar lowercase em todas as colunas;**


In [0]:
#Função para lowercase em DataFrames
def columns_lower(df):
    #Transforma todas colunas em minusculo
    df.columns = df.columns.str.lower()
    return df.head()

display(columns_lower(df_insurance))


## 2) **Excluir caracteres especiais dos nomes das colunas;**




In [0]:
#Função para limpar colunas
def column_clear(df):
    #Tira os espaços e insere underline
    df.columns = df.columns.str.replace(' ', '_')
    #Tira os caracteres especiais
    df.columns = df.columns.str.replace(r'[^0-9a-zA-Z_]+','', regex = True)
    return df.head()

column_clear(df_insurance)


⚠️ Não temos caracteres especiais nos nomes das colunas neste DataFrame

## 3) **Tratamento dos outliers;**

- Maioria das nossas variáveis são categóricas, vou selecionar somente as numéricas 



In [0]:

df_insurance[['credit_score', 'annual_mileage','speeding_violations', 'duis', 'past_accidents']].describe()

- (Decidi por não tratar "outliers" de 'speeding_violations', 'duis', 'past_accidents' pois essas variáveis representam comportamentos raros, mas extremamente relevantes para prever sinistros.)

In [0]:
#Selecionando as colunas que iremos verificar
df_outlier = df_insurance[['credit_score', 'annual_mileage']]
display(df_outlier.head())

- Normalizando as variáveis

In [0]:
sns.histplot(df_outlier['credit_score'])
sns.histplot(df_outlier['annual_mileage'])

In [0]:
#Normalizando as variáveis
df_outlier = StandardScaler().fit_transform(df_outlier)

#Mostrando as variáveis normalizadas
df_outlier = pd.DataFrame(df_outlier, columns=['credit_score', 'annual_mileage'])
display(df_outlier.head())

In [0]:
sns.histplot(df_outlier)

- Usando o método Box-Plot para identificar possíveis Outliers nas variáveis contínuas

In [0]:
def mostrar_boxplot(df):
    '''
    DF tem que possuir somente as colunas que serão plotadas
    '''
    
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=df) 
    plt.xticks(rotation=45)      
    plt.show()

mostrar_boxplot(df_outlier)


- As variáveis **possuem** Outliers, vou tráta-los.

In [0]:
df = df_insurance.copy()


#Coluna 'credit_score'
q1 = np.nanpercentile(df['credit_score'], 25) #0.4171
q3 = np.nanpercentile(df['credit_score'], 75) #0.6183

#Calculando os limites
lim_inf = q1 - 1.5 * (q3 - q1) #0.1155
lim_sup = q3 + 1.5 * (q3 - q1) #0.9199

outliers = len(df[df['credit_score'] > lim_sup]) + len(df[df['credit_score'] < lim_inf])

#Valores que eram Outliers agora tem o valor exato do limite
df['credit_score'] = df['credit_score'].clip(lower=lim_inf, upper=lim_sup)

sns.boxplot(data=df['credit_score'])
print(f"Outliers tratados: {outliers}")

In [0]:
#Coluna 'annual_mileage'
q1 = np.nanpercentile(df['annual_mileage'], 25) #10000.0
q3 = np.nanpercentile(df['annual_mileage'], 75) #14000.0

#Calculando os limites
lim_inf = q1 - 1.5 * (q3 - q1) #4000.0
lim_sup = q3 + 1.5 * (q3 - q1) #20000.0

outliers = len(df[df['annual_mileage'] > lim_sup]) + len(df[df['annual_mileage'] < lim_inf])

#Valores que eram Outliers agora tem o valor exato do limite
df['annual_mileage'] = df['annual_mileage'].clip(lower=lim_inf, upper=lim_sup)

sns.boxplot(data=df['annual_mileage'])
print(f"Outliers tratados: {outliers}")

## 4) **Tratamento dos missing values;**

- Verificando quantos missing values temos e em quantas colunas:

In [0]:
df.isna().sum()

**credit_score** (982) e **annual_mileage** (957) são as variáveis que teremos que tratar os missing values 



In [0]:
def mostra_missing_value(df):
    #Mostra a quantidade de NaN por coluna em ordem descrescente
    total = df.isnull().sum()

    #Divide qnt de NaN pela quantidade de nulos e não nulos * 100
    percent = 100*round((df.isnull().sum()/df.isnull().count()),2)

    #Faz uma tabela com a quantidade de NaN e a porcentagem que representa
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data)

mostra_missing_value(df[['credit_score','annual_mileage']])

Vimos que as colunas seguem a distribuição normal. Por isso, vou optar por substituir os missing values pela média.

In [0]:
#Descobrindo os indices de missing values
cs_indices = df['credit_score'].isna()
am_indices = df['annual_mileage'].isna()

#Descobrindo a média das colunas 
cs_mean = df['credit_score'].sum() / df['credit_score'].count()
am_mean = df['annual_mileage'].sum() / df['annual_mileage'].count()

#Atribuindo as médias aos missing values
df.loc[cs_indices, 'credit_score'] = cs_mean
df.loc[am_indices, 'annual_mileage'] = am_mean

mostra_missing_value(df[['credit_score','annual_mileage']])

## 5) **Lidar com dados categóricos;**

In [0]:
df.head()

**Variáveis categóricas:**

- **Ordinais**: age, driving_experience, education, income
- **Nominais**: gender, race, vehicle_year, vehicle_type

**Vou usar uma abordagem diferente para cada tipo.**

- Tratando as ordinais com **OrdinalEncoding**

In [0]:
#Listando cada valor que será substituído por um valor númerico ordinal
l_ages = [['16-25', '26-39', '40-64', '65+']]
l_exp = [['0-9y', '10-19y', '20-29y', '30y+']]
l_education = [['none', 'high school', 'university']]
l_income = [['poverty', 'working class', 'middle class', 'upper class']]

#Adicionando as listas em uma lista só
categories = [l_ages, l_exp, l_education, l_income]

#Criando o DF que contém todas as colunas que vamos tratar
df_ord = df[['age', 'driving_experience', 'education', 'income']]

#Loop iterando por todas as colunas
for cat, col in zip(categories, df_ord.columns):
    #Instanciando o encoder
    enc = OrdinalEncoder(categories=cat)
    #Atribuindo as colunas transformadas no DF original
    df.loc[:,col] = enc.fit_transform(df[[col]])

In [0]:
df[['age', 'driving_experience', 'education', 'income']].head()

- Tratando as nominais com **Dummies**


In [0]:
#Lista com colunas que serão tratadas
l_columns = ['gender', 'race', 'vehicle_year', 'vehicle_type']

#Iterando sobre cada coluna, criando dummies e concatenando
for column in l_columns:

  #Criando a variável Dummie para a coluna atual com valor 0 ou 1
  dummies = pd.get_dummies(df[column], dtype=int)
  df = pd.concat([df, dummies], axis = 1) #Concatenando
  df.drop(columns = [column], inplace=True) #Exclui a coluna original

df.head()

In [0]:
column_clear(df)

- ####  Salvando na camada Silver

In [0]:
spark_df = spark.createDataFrame(df)

spark_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("workspace.silver.t_car_insurance_claim")

##6) **Fazer EDA (Análise Expploratória de Dados);**

Analisando o DataFrame

In [0]:
df_insurance.info()

Estatísticas descritivas das colunas

In [0]:
df_insurance.describe()

Analisando Correlação de variáveis

In [0]:
#Selecionando todas colunas numéricas
df_numeric = df[['age', 'driving_experience', 'education', 'income', 'annual_mileage', 'speeding_violations', 'duis', 'past_accidents', 'outcome']]

plt.figure(figsize=(10, 7))
sns.heatmap(df_numeric.corr(), cmap='CMRmap', annot = True)
plt.show()

Interpretações

- Aparentemente, pessoas com maior renda tendem a andar menos com o carro no ano (correlação negativa entre income e annual_mileage).
- Quem dirige mais por ano tende a registrar menos multas por excesso de velocidade, porém essa relação pode ser influenciada por idade e experiência do motorista.
- Quanto maior a experiência ao volante, maior a quantidade acumulada de violações de velocidade e acidentes passados — o que pode ser explicado pelo maior tempo de exposição ao trânsito.
- Quanto maior a idade, menos a chance de sinistros.

Embora se espere que motoristas experientes tenham menos risco, os dados mostram o contrário. Possivelmente porque motoristas experientes têm mais anos dirigindo e, portanto, mais tempo para acumular infrações e acidentes.

In [0]:
df_insurance.head()

- Qual gênero tem a maior frequência de sinistros?

In [0]:
plt.figure(figsize=(6,4))
sns.countplot(data=df_insurance, x=df_insurance['gender'], hue='outcome')

Aparentemente a proporção é igual.

- Verificando por faixa-etária:

In [0]:
#Filtrando sexos
df_fem = df_insurance[df_insurance['gender'] == 'female']
df_mas = df_insurance[df_insurance['gender'] == 'male']

#Plotando gráfico para mulheres
plt.figure(figsize=(6,4))
sns.countplot(data=df_fem, x='age', hue='outcome', order=['16-25', '26-39', '40-64', '65+'])
plt.title('Mulheres')

#Plotando gráfico para homens
plt.figure(figsize=(6,4))
plt.title('Homens')
sns.countplot(data=df_mas, x='age', hue='outcome', order=['16-25', '26-39', '40-64', '65+'])

plt.show()



Em ambos os gêneros, temos uma queda na quantidade de sinistros com o decorrer da idade. Isso confirma a matriz de correlação entre 'age' e 'outcome'

- Grafico de sexo, idade e se nao tem ou tem filho

In [0]:
# Contagem por faixa etária
plt.figure(figsize=(6,4))
sns.countplot(data=df_fem, x='age', hue='children', order=['16-25', '26-39', '40-64', '65+'])
plt.title('Mulheres')

#Plotando gráfico para homens
plt.figure(figsize=(6,4))
sns.countplot(data=df_mas, x='age', hue='children', order=['16-25', '26-39', '40-64', '65+'])
plt.title('Homens')

Enquanto há uma queda de sinistros ao decorrer da idade, há um aumento na quantidade de filhos em ambos os sexos. Isso mostra que o aumento na quantidade de filhos **não** tem uma relação com o aumento de sinistros.

Uma possível conclusão pode ser: Pessoas com filhos podem ter mais cuidado no volante.

- Violações de velocidade por idade e sexo

In [0]:
#Plotando o gráfico das mulheres
plt.figure(figsize=(10,7))
plt.title('Mulheres')
sns.countplot(data=df_fem, x='age', hue='speeding_violations', order=['16-25', '26-39', '40-64', '65+'])


#Plotando o gráfico dos homens
plt.figure(figsize=(10,7))
plt.title('Homens')
sns.countplot(data=df_mas, x='age', hue='speeding_violations', order=['16-25', '26-39', '40-64', '65+'])

Comparando com o gráfico dos filhos, isso pode explicar o motivo do crescimento das infrações serem bem nas idades onde pais costumam emprestar seus carros para seus filhos.

##7) **Seleção de features;**


- Seleção de Features usando KBest e teste chi2

In [0]:
# Selecionando features e target
df2 = df.drop(columns=['id', 'credit_score', 'postal_code', 'annual_mileage', 'speeding_violations', 'duis','past_accidents'])
X = df2.drop(columns=['outcome'])
y = df2['outcome']

# Selecionando as melhores features com o KBest
chi2_seletor = SelectKBest(chi2, k='all')
chi2_seletor.fit(X,y)

#DataFrame com as informações de Chi2 e P-value para cada feature
pd.DataFrame({
    'Feature': X.columns,
    'Chi2 Score': chi2_seletor.scores_,
    'p-value': np.round(chi2_seletor.pvalues_, 2)
})

Teste mostra claramente que 'majority', 'minority', 'sedan' e 'sports car' **NÃO** são importantes para o nosso modelo.

- Análise de Correlação

In [0]:
df_corr = df.drop(columns=['id'])

plt.figure(figsize=(15, 10))
sns.heatmap(round(df_corr.corr().abs(), 2), cmap='CMRmap', annot = True)
plt.show()

**postal_code, sedan, sports_car, majority e minority** tem baixíssima correlação com a variável target e com as outras features. 

As features que mais fortemente se correlacionam são: 'age' e 'drive_experience'. Faz sentido.

- Seleção de features com f_classif (ANOVA)

In [0]:
# Selecionando features
X2 = df[['credit_score', 'postal_code', 'annual_mileage', 'speeding_violations', 'duis','past_accidents']]

# Selecionando as melhores features com o KBest
selector = SelectKBest(score_func= f_classif, k='all')
selector.fit(X2, y)

#DataFrame com as informações de Chi2 e P-value para cada feature
pd.DataFrame({
    'Feature': X2.columns,
    'F-Score': selector.scores_,
    'p-value': selector.pvalues_
})

Teste mostra que a variável 'postal_code' pode ser descartada. Tem um F-Score baixo, e p-value mais alto.

- Seleção de Features com XGBoost

In [0]:
df.dtypes

In [0]:
# Selecionando as features
X = df.drop(columns=['id', 'outcome'])

#Colunas estão como object, XGBoost não aceita. Convertendo para 'int'
X = X.astype({
    'age': 'int64',
    'driving_experience': 'int64',
    'education': 'int64',
    'income': 'int64'
})


# Treinando o modelo
model = XGBClassifier(random_state = 1)
model.fit(X, y)

# Criando um DataFrame com 
importances = model.feature_importances_
feat_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': np.round(importances, 4)
})

feat_importances

- O XGBoost confirma o que as outras medidas de Feature Selection confirmaram.

Além do **id**, vou remover do modelo as colunas: **'postal_code', 'majority', 'minority', 'sedan' e 'sports car'**

In [0]:
df.drop(columns= ["id", "postal_code", "majority", "minority", "sedan", "sports_car"], inplace=True)
df.head()

- #### Salvando na camada Gold

In [0]:
spark_df = spark.createDataFrame(df)

spark_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("workspace.gold.t_car_insurance_claim")

## 8) **Definir as amostras de treinamento e validação;**


In [0]:
X = df.drop(columns=['outcome'])

#Colunas estão como object, XGBoost não aceita. Convertendo para 'int'
X = X.astype({
    'age': 'int64',
    'driving_experience': 'int64',
    'education': 'int64',
    'income': 'int64'
})

y = df['outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 20111974)

## 9) **Escolher o algoritmo a ser aplicado;**
## 10) **Métricas de performance dos algoritmos;**
## 11) **Selecionar o melhor modelo preditivo;**

- Testando algoritmos, verificando métricas de performance e selecionamento o melhor

DecisionTreeClassifier, Random Forest, K-Nearest Neighbors (KNN), XGBoost

- **DecisionTree Classifier**

In [0]:
#Criando e treinando o modelo
clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#Medindo a performance
print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

print(f"\nMatriz de Confusão:\n{confusion_matrix(y_test, y_pred)}")

**DecisionTree Classifier**
- Acurária: 0.757
- Recall: 0.634
- F1-Score: 0.622
- MAE: 0.243

- **Random Forest Classifier**

In [0]:
#Criando e treinando o modelo
rf_clf = RandomForestClassifier(n_estimators=100, random_state=1)
rf_clf.fit(X_train, y_train)
y_pred2 = rf_clf.predict(X_test)

#Medindo a performance
print(f"Acurácia: {accuracy_score(y_test, y_pred2)}")
print(f"Recall: {recall_score(y_test, y_pred2)}")
print(f"F1-Score: {f1_score(y_test, y_pred2)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred2)}")

print(f"\nMatriz de Confusão:\n{confusion_matrix(y_test, y_pred2)}")

**RandomForest Classifier**
- Acurária: 0.83
- Recall: 0.69
- F1-Score: 0.72
- MAE: 0.17

- **KNeighbors Classifier**

In [0]:
#Padronizando variáveis continuas para o melhor funcionamento do modelo
X_scaled = X.copy()
col_scale = ['credit_score', 'annual_mileage', 'speeding_violations', 'duis', 'past_accidents']
Scaler = StandardScaler()

X_scaled[col_scale] = Scaler.fit_transform(X_scaled[col_scale])
X_scaled.head()

In [0]:
# Separando os conjuntos de treino e teste escalados
X_sc_train, X_sc_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state= 20111974)

#Criando e treinando o modelo
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_sc_train, y_train)
y_pred3 = knn.predict(X_sc_test)

#Medindo a performance
print(f"Acurácia: {accuracy_score(y_test, y_pred3)}")
print(f"Recall: {recall_score(y_test, y_pred3)}")
print(f"F1-Score: {f1_score(y_test, y_pred3)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred3)}")

print(f"\nMatriz de Confusão:\n{confusion_matrix(y_test, y_pred3)}")

**KNeighbors Classifier**
- Acurária: 0.812
- Recall: 0.679
- F1-Score: 0.694
- MAE: 0.188

 - **XGBoost Classifier**

In [0]:
X.dtypes

In [0]:
# Criando e treinando o modelo
xgb = XGBClassifier(random_state = 1, learning_rate=0.03, n_estimators = 150)
xgb.fit(X_train, y_train)
y_pred4 = xgb.predict(X_test)

#Medindo a performance
print(f"Acurácia: {accuracy_score(y_test, y_pred4)}")
print(f"Recall: {recall_score(y_test, y_pred4)}")
print(f"F1-Score: {f1_score(y_test, y_pred4)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred4)}")

print(f"\nMatriz de Confusão:\n{confusion_matrix(y_test, y_pred4)}")

**XGBoost Classifier**

- Acurácia: 0.84
- Recall: 0.74
- F1-Score: 0.74
- MAE: 0.16

## Modelo preditivo selecionado: **XGBoost** ✅

- Nem todos modelos tiverem bons resultados de métricas. Mas sem dúvidas o **XGBoost** foi o melhor pois teve um ótimo balanceamento com **84% de acurácia** e com poucos erros. Além de ser um modelo rápido e robusto. 

No nosso caso de previsão de sinistros, as métricas de Recall e F1-Score são importantes e foram muitos boas neste modelo, com **Recall de 0.74** e **F1-Score de 0.74**, indicando que é bom em detectar sinistros reais e controle de falsos positivos.